In [1]:
import random
import numpy as np
import math
import copy
import matplotlib.pyplot as plt
import pickle
from datetime import datetime
random.seed(datetime.now())

# Basic Tool

In [2]:
def array2int(array):
    tmp = 0
    for idx in range(array.size):
        tmp = tmp + 2**idx*array[idx]
    return tmp

# Error Correction Code Method

## Build Up $GF(2^8)$ Table

In [3]:
def Build_GF_log_Table_with_GF_exp_Table(GF_a_exp_Table_int):
    log_Table = {int(value):int(key) for key,value in GF_a_exp_Table_int.items()}
    # Since log_Table could not find log(0) , we define log(0) would be none in exponenet form
    log_Table[0]=None
    return log_Table

### GF_mult()

### result = $\alpha^{x\_exp}\alpha^{y\_exp}=\alpha^{(x\_exp+y\_exp)\%(2^n-1)}$

In [4]:
def GF_mult_tb(x,y,n,GF_log_a_Table_int):# Verified
    n = int(n)
    #  x,y in GF(2^n)
    
    if x == 0 or y == 0 :# for 0 case
        return 0
        
    modulus = (1<<n)-1
    x_exp = GF_log_a_Table_int[x]
    y_exp = GF_log_a_Table_int[y]
    
    result_exp = (x_exp + y_exp)%(modulus) # Because x^(2**n-1) = 1, so (2**n -1) equivalent to 0
    return GF_a_exp_Table_int[result_exp]

In [5]:
def GF_Table_Generation_with_irr(alpha_int,irr_int):
    table = {}
    for i in range(255):
        table[i]= GF_2_8_exp(alpha=alpha_int,exp=i,irr_int=irr_int)
    return table

# Cryptography Method

# AES Galois Field Polynomial:
### $GF(2^8)$=$\frac{F_2[x]}{irr(x)}$
## $$irr(x) = 1+x+x^3+x^4+x^8$$
    

In [6]:
n=8
AES_irr_arr = np.asarray([1,1,0,1,1,0,0,0,1])
#                         0 1 2 3 4 5 6 7 8
AES_irr_int = array2int(AES_irr_arr)

AES_irr_list = [1,1,0,1,1,0,0,0,1]

## Do the Exponentiaition in $GF(2^8)$

In [7]:
def GF_irr_exp(alpha,exp,irr_int,n):
    tmp = 1
    for i in range(1,exp+1):
        tmp = GF_mult_irr(alpha,tmp,irr_int,n)
    return tmp

## Do the Multiplication in $GF(2^8)$

### Poly_Mult_GF()

In [8]:
def Poly_Mult_lst(poly1,poly2):
    poly1_size = len(poly1)
    poly1_deg  = poly1_size-1
    poly2_size = len(poly2)
    poly2_deg  = poly2_size-1
    
    result = [0]*(poly1_size+poly2_size-1)
    
    for idx1 in range(poly1_size):
        for idx2 in range(poly2_size):
            current_idx = idx1 + idx2
            result[current_idx]= result[current_idx] ^ (poly1[idx1]&poly2[idx2])
    return result    

In [9]:
def GF_mult_irr_list(poly1_lm,poly2_lm,irr_poly_lm):
    
    poly1_arr = np.asarray(poly1_lm)
    poly2_arr = np.asarray(poly2_lm)
    irr_arr   = np.asarray(irr_poly_lm)[::-1]
    
    poly1_size = len(poly1_lm)
    poly1_deg  = poly1_size-1
    poly2_size = len(poly2_lm)
    poly2_deg  = poly2_size-1
    
    result = np.asarray([0]*(poly1_size+poly2_size-1))
    
    for idx1 in range(poly1_size):
        for idx2 in range(poly2_size):
            current_idx = idx1 + idx2
            result[current_idx]= result[current_idx] ^ (poly1_arr[idx1] & poly2_arr[idx2])
            
    result = result[::-1]# from highest degree to lowest
    #print("remainder:",result)
            
    for idx,val in enumerate(result):
        if idx > (result.size - irr_arr.size):
            break
        if val == 1:
            result[idx:idx+irr_arr.size] = result[idx:idx+irr_arr.size] ^ irr_arr
            #print("remainder:",result)
            #print("divisor:",irr_arr)
            
    return result[::-1] # lsb_msb

In [10]:
def GF_mult_irr(x,y,irr,n):
    
    poly1    = [int(val) for val in "{0:08b}".format(x) ][::-1][:n]
    poly2    = [int(val) for val in "{0:08b}".format(y) ][::-1][:n]
    irr_poly = [int(val) for val in "{0:08b}".format(irr) ][::-1][:n+1]
#    print("poly1",poly1)
#    print("poly2",poly2)
#    print("irr_poly",irr_poly)
    
    poly1xpoly2_lst = GF_mult_irr_list(poly1_lm=poly1,poly2_lm=poly2,irr_poly_lm=irr_poly)
            
            
    return array2int(poly1xpoly2_lst)

# Polynomial Divider

In [11]:
def GF_divide_irr_int(x,irr,n):
    poly1   = [int(val) for val in "{0:08b}".format(x) ][8-n:]
    irr_poly= [int(val) for val in "{0:08b}".format(irr) ][8-n:]
    
    poly1_arr = np.asarray(poly1)
    irr_arr   = np.asarray(irr_poly)
    
    poly1_size = len(poly1)
    poly1_deg  = poly1_size-1
    
    remainder = copy.deepcopy(poly1_arr)
    quotient  = []
            
    for idx,val in enumerate(remainder):
        if (poly1_deg-idx) <= 7:
            break
        if val == 1:
            #print("remainder:",result)
            #print("divisor  :",irr_arr)
            quotient.append(1)
            remainder[idx:idx+9] = remainder[idx:idx+9] ^ irr_arr
            
    quotient =  np.asarray(quotient)[::-1]
            
    return array2int(remainder[::-1])

In [12]:
def Binary_divide_poly_lst(dividend_l_m,divisor_l_m):
    
    dividend_arr = np.asarray(Poly_decorator(dividend_l_m))[::-1]# turn to msb -> lsb
    divisor_arr  = np.asarray(Poly_decorator(divisor_l_m))[::-1]
    
    divisor_deg  = divisor_arr.size - 1
    divisor_size = divisor_arr.size
    
    
    remainder = copy.deepcopy(dividend_arr)
    quotient  = []
            
    for idx,val in enumerate(remainder):
        if (dividend_arr.size - idx) <= divisor_deg:
            break
        
        if val == 1:
            quotient.append(1)
            remainder[idx:idx+divisor_size] = remainder[idx:idx+divisor_size] ^ divisor_arr
        else:
            quotient.append(0)
            
    remainder = Poly_decorator(remainder[::-1])
            
    return remainder.tolist(),quotient[::-1]

# Polynomial Addition in GF

In [13]:
def Poly_Add_GF_lst(poly1_l_m,poly2_l_m):#Verified
    if len(poly1_l_m) > len(poly2_l_m):
        large_poly = poly1_l_m
        small_poly = poly2_l_m
    else:
        large_poly = poly2_l_m
        small_poly = poly1_l_m
        
    result = large_poly
    for idx,val in enumerate(small_poly):
        result[idx] = result[idx] ^ small_poly[idx]
    return result

In [14]:
def GF_order(alpha,irr_int,n):    
    for i in range(1,2**n):
        if GF_irr_exp(alpha=alpha,exp=i,irr_int=irr_int,n=n) == 1:
            print("Ord(",alpha,") = ",i)
            return i

In [15]:
def poly_deg(poly):
    # return the degree of polynomial
    size = len(poly)
    if size == 0:
        return None
    
    for idx in range(size-1,-1,-1):
        if poly[idx] == 0:
            continue
        else:
            return idx
    return 0

In [16]:
def Poly_decorator(poly):
    # truncate the redundant zero in higher degree
    size = len(poly)
    for i in range(size-1,-1,-1):
        if poly[i] == 0:
            continue
        else:
            return poly[:i+1]
    return [0]

In [17]:
def Evaluation(Poly,sub_val,irr,n):
    result = 0
    for exp,val in enumerate(Poly):
        if val == 1:# Do the Exponentiation
            tmp = 1
            for itr in range(exp):
                tmp = GF_mult_irr(tmp,sub_val,irr,n)
            result = result ^ tmp
    return result

### GF_inv()
- Extended Euclidean Version

In [18]:
def Extendend_Euclidean(modulus_lst_l_m,alpha_lst_l_m):# Error Overhere
    # as + bt = c
    a = modulus_lst_l_m
    b = alpha_lst_l_m
    r_prev = a
    r_cur  = b
    s_prev = [1]
    s_cur  = [0]
    t_prev = [0]
    t_cur  = [1]
    itr = 0
    
    while poly_deg(r_cur) >= 1 :
        
        itr = itr +1
        rem, q = Binary_divide_poly_lst(dividend_l_m=r_prev,divisor_l_m=r_cur)
        
        #print("r_prev =",r_prev)
        #print("q*r_cur + rem =",Poly_Add_GF_lst(Poly_Mult_lst(r_cur,q),rem))
        
        r_cur, r_prev = rem, r_cur
        
        s_cur,s_prev  = Poly_Add_GF_lst( s_prev ,Poly_Mult_lst(poly1=q,poly2=s_cur)) , s_cur
        t_cur,t_prev  = Poly_Add_GF_lst( t_prev ,Poly_Mult_lst(poly1=q,poly2=t_cur)) , t_cur
        
        #print("modulus*s + test*t =",Poly_Add_GF_lst(Poly_Mult_lst(s_cur,modulus_lst_l_m),Poly_Mult_lst(test,t_cur)))
        #print("rem =", r_cur)
        
    # modulus * s_cur +  alpha_lst * t_cur = r_cur
    return s_cur , t_cur, r_cur

In [19]:
test=[1, 1, 1, 0, 0, 0, 0, 0]

In [20]:
s,t,r = Extendend_Euclidean(modulus_lst_l_m=AES_irr_list,alpha_lst_l_m=test)

In [21]:
Binary_divide_poly_lst(Poly_Mult_lst(test,t),AES_irr_list)

([1], [0, 1])

In [22]:
test_inv =t

# Find all the roots of AES Irreducible Polynomial:

# AES Galois Field Polynomial:
### $GF(2^8)$=$\frac{F_2[x]}{irr(x)}$
## $$irr(x) = 1+x+x^3+x^4+x^8$$
    

# Test AES

In [23]:
S_Matrix_lsb_msb = np.asarray( [
[1,0,0,0,1,1,1,1],
[1,1,0,0,0,1,1,1],
[1,1,1,0,0,0,1,1],
[1,1,1,1,0,0,0,1],
[1,1,1,1,1,0,0,0],
[0,1,1,1,1,1,0,0],
[0,0,1,1,1,1,1,0],
[0,0,0,1,1,1,1,1]])

offset_lsb_msb = np.asarray([1,1,0,0,0,1,1,0])

In [24]:
s,inverse_7,r = Extendend_Euclidean(modulus_lst_l_m=AES_irr_list,alpha_lst_l_m=[1,1,1,0,0,0,0,0])
inverse_7 = np.asarray(inverse_7)

In [25]:
(S_Matrix_lsb_msb.dot(inverse_7)+offset_lsb_msb)%2

array([1, 0, 1, 0, 0, 0, 1, 1])

In [26]:
x13x11x9x8x6x5x4x31=[1,0,0,1,1,1,1,0,1,1,0,1,0,1]
x6x4x2x1=[1,1,1,0,1,0,1]
x7x1=[1,1,0,0,0,0,0,1]

In [27]:
GF_divide_irr_int(11129,283,n)

193

In [28]:
Binary_divide_poly_lst(x13x11x9x8x6x5x4x31,AES_irr_list)

([1, 0, 0, 0, 0, 0, 1, 1], [0, 0, 0, 1, 0, 1])

In [29]:
Poly_Mult_lst(x6x4x2x1,x7x1)

[1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1]

#  AES S-BOX Research

### $R(z) = z^8+z^4+z^3+z+1$

In [30]:
AES_irr_arr = np.asarray([1,1,0,1,1,0,0,0,1])
#                         0 1 2 3 4 5 6 7 8
AES_irr_int = array2int(AES_irr_arr)

AES_irr_list = [1,1,0,1,1,0,0,0,1]

In [31]:
for i in range(256):
    if Evaluation(Poly=AES_irr_list,sub_val=i,irr=AES_irr_int,n=8) == 0:
        print("Root Found :", i)

Root Found : 2
Root Found : 4
Root Found : 16
Root Found : 27
Root Found : 77
Root Found : 94
Root Found : 228
Root Found : 250


In [32]:
print("ord(",2  ,")",GF_order(2  ,AES_irr_int,8))
print("ord(",4  ,")",GF_order(4  ,AES_irr_int,8))
print("ord(",16 ,")",GF_order(16 ,AES_irr_int,8))
print("ord(",27 ,")",GF_order(27 ,AES_irr_int,8))
print("ord(",77 ,")",GF_order(77 ,AES_irr_int,8))
print("ord(",94 ,")",GF_order(94 ,AES_irr_int,8))
print("ord(",228,")",GF_order(228,AES_irr_int,8))
print("ord(",250,")",GF_order(250,AES_irr_int,8))

Ord( 2 ) =  51
ord( 2 ) 51
Ord( 4 ) =  51
ord( 4 ) 51
Ord( 16 ) =  51
ord( 16 ) 51
Ord( 27 ) =  51
ord( 27 ) 51
Ord( 77 ) =  51
ord( 77 ) 51
Ord( 94 ) =  51
ord( 94 ) 51
Ord( 228 ) =  51
ord( 228 ) 51
Ord( 250 ) =  51
ord( 250 ) 51


# $Q(y) = y^4+y+1$

In [33]:
q_irr_arr = np.asarray([1,1,0,0,1])
#                       0 1 2 3 4 
q_irr_int = array2int(q_irr_arr)
q_irr_list = [1,1,0,0,1]

In [34]:
for i in range(2**4):
    if Evaluation(Poly=q_irr_list,sub_val=i,irr=q_irr_int,n=4) == 0:
        print("Root Found :", i)

Root Found : 2
Root Found : 3
Root Found : 4
Root Found : 5


# $P(x) = x^2 + x + w^{14}$

In [35]:
w14 = GF_irr_exp(alpha=2,exp=14,irr_int=q_irr_int,n=4)

In [38]:
p_irr_arr  = np.asarray([w14,1,1])
p_irr_list = [w14,1,1]

In [ ]:
def CF_mult(poly1_l_m,poly2_l_m,p_irr_list ,q_irr_int):
    
    poly_1_l_m_arr = np.asarray(poly1_l_m)
    poly_2_l_m_arr = np.asarray(poly2_l_m)
    poly_1_size = poly_1_l_m_arr.size
    poly_2_size = poly_2_l_m_arr.size
    
    result = np.asarray([0]*(poly_1_size+poly_2_size-1))
    
    for idx1 in range(poly_1_size):
        for idx2 in range(poly_2_size):
            cur_idx = idx1 + idx2
            result[cur_idx] = result[cur_idx] ^ (GF_mult_irr(x=poly_1_l_m_arr[idx1],y=poly_2_l_m_arr[idx2],irr=q_irr_int,n=4))
            
            
    result = Poly_decorator(result)
    result = result[::-1] # Reversal to MSB  to LSB 
    
    
    # Start the division
    dividend = copy.deepcopy(result)
    divisor  = np.asarray(p_irr_list)[::-1]
    
    for idx,val in enumerate(dividend):
        Extendend_Euclidean(modulus_lst_l_m= q_irr_list, alpha_lst_l_m=)
        
    
    
    
    
    
    
            